This notebook is a - kind of - continuation from: inside_bar_explore.ipynb

In [22]:
import pandas as pd
import plotly.graph_objects as go
import utils
import datetime as dt
from dateutil.parser import *

In [23]:
df_trades = pd.read_csv("./Data/Pairs/USD_JPY_H4_trades.csv")

In [24]:
pair = "USD_JPY"
granularity = "M5"

In [25]:
df_raw = pd.read_csv(utils.get_hist_data_filename(pair, granularity))
df_raw.shape

(223553, 16)

In [26]:
non_nums = ['ticker', 'time', 'volume']
num_cols = [x for x in df_raw.columns if x not in non_nums]
df_raw[num_cols] = df_raw[num_cols].apply(pd.to_numeric)

In [27]:
df_trades["time"] = [parse(x) for x in df_trades.time]
df_raw["time"] = [parse(x) for x in df_raw.time]

We need to know the time of the next trade (shift), and the time of the last 5 minute trade in the 4 hour window (because the "H4" granularity data is what we're digging into).  So, that's why we're adding 03:55 to the start time of the 4 hour window.

In [28]:
df_trades["next"] =  df_trades["time"].shift(-1)
df_trades["trade_end"] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades["trade_start"] = df_trades.time + dt.timedelta(hours=4)
df_trades[["time", "next", "trade_end", "trade_start"]].head()

,time,next,trade_end,trade_start
0,2020-01-05 22:00:00+00:00,2020-01-06 22:00:00+00:00,2020-01-07 01:55:00+00:00,2020-01-06 02:00:00+00:00
1,2020-01-06 22:00:00+00:00,2020-01-07 06:00:00+00:00,2020-01-07 09:55:00+00:00,2020-01-07 02:00:00+00:00
2,2020-01-07 06:00:00+00:00,2020-01-09 18:00:00+00:00,2020-01-09 21:55:00+00:00,2020-01-07 10:00:00+00:00
3,2020-01-09 18:00:00+00:00,2020-01-16 10:00:00+00:00,2020-01-16 13:55:00+00:00,2020-01-09 22:00:00+00:00
4,2020-01-16 10:00:00+00:00,2020-01-20 14:00:00+00:00,2020-01-20 17:55:00+00:00,2020-01-16 14:00:00+00:00


In [29]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [30]:
def signal_text(signal):
    if signal == 1:
        return "BUY"
    elif signal == -1:
        return "SELL"
    else:
        return "NONE"
    


#### triggered function:  Receive Buy/Sell direction, get current price from a finer granularity against higher granularity entry(signal) price.

Simplified example:
* We have a Entry Price of 1.00, identified in a 4 hour candle
* We meet 12 5 minute candles at < 1.00
* On candle 13 (an hour in) we get a price of 2.99
* We buy at 2.99, but our take profit is 3.00 - so this is a worthless trade.
* Alternately, candle 13 had a price of 1.01.  If we hit our take profit, we done well. 

In [31]:
def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    else:
        return False

In [32]:
def process_buy(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price >= TP:
            return 2.0
        elif price <= SL:
            return -1.0
    return 0.0

def process_sell(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price <= TP:
            return 2.0
        elif price >= SL:
            return -1.0
    return 0.0


def process_trade(start_index, direction, TP, SL, prices, start_price):
    if direction == 1:
        return process_buy(start_index, TP, SL, prices, start_price)
    elif direction == -1:
        return process_sell(start_index, TP, SL, prices, start_price)


We've used iterrows to loop through DataFrames until now, but this can be slow.  The process_m5 function presents a faster alternative.

In [41]:
def process_m5(m5_df, row):
    result = 0.0
    for index, price in enumerate(m5_df.mid_c.values):
        if triggered(row.SIGNAL, price, row.ENTRY) == True:
            # print(f"   {signal_text(row.SIGNAL)} Signal at (index {index}) {price: 2f} from Entry: {row.ENTRY: 2f} ") 
            result = process_trade(index, row.SIGNAL, row.TAKEPROFIT, row.STOPLOSS, m5_df.mid_c.values, row.ENTRY)          
            break
    
    return  result


### Important 

Remember at this this point in this notebook:  
1. df_trades is built on h4 granularity (4 hour candles)
2. df_raw is built on m5 granularity (5 minute candles)

This is really obvious while working through a tutorial, but consider putting granularity in DF naming in future.

In [42]:
total = 0
for index, row in df_trades.iterrows():
    m5_data = df_raw[(df_raw.time >= row.trade_start) & (df_raw.time <= row.trade_end)]
    # print(f"{index} {row.time} {signal_text(row.SIGNAL)} {row.ENTRY: 2f} {m5_data.shape}")
    total += process_m5(m5_data, row)
    # if index > 10:
    #     break
print(f"Aggregated total on 2 vs -1 basis:  {total}")

Aggregated total on 2 vs -1 basis:  138.0


In [ ]:
m5_data.shape

(335, 16)